In [1]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import BatchNormalization,Dense,Dropout,GlobalAveragePooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adamax

def load_data(path, annotations):
    images = []
    labels = []

    with open(annotations, 'r') as file:
        annotations = json.load(file)

    for annotation in annotations:
        img = keras.preprocessing.image.load_img(
            os.path.join(path, f'image_id_{annotation["image_id"]:03}.jpg'),
            target_size=(224, 224)
        )
        img_array = keras.preprocessing.image.img_to_array(img)
        images.append(img_array)

        labels.append(annotation['category_id'] - 1)

    return np.array(images), np.array(labels)

# Load training and validation data
train_images, train_labels = load_data('train/train', 'train_annotations.json')
valid_images, valid_labels = load_data('valid/valid', 'valid_annotations.json')


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adamax
from kerastuner.tuners import RandomSearch

class_count=2
lr= 0.001
img_shape=(224,224,3)
epochs = 10

train_labels = to_categorical(train_labels, num_classes=class_count)
valid_labels = to_categorical(valid_labels, num_classes=class_count)

datagen = ImageDataGenerator(
        rotation_range=45,
        zoom_range = 0.2,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

datagen.fit(train_images)

def build_model(hp):
    base_model = tf.keras.applications.EfficientNetV2B0(
        weights = "imagenet",
        include_top = False,
        input_shape = img_shape
    )
    base_model.trainable=True

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        Dropout(rate=0.2, seed = 42),
        layers.Dense(
            units=256,
            activation='relu',
            kernel_regularizer = regularizers.l2(l=hp.Float('l2_reg', min_value=0.015, max_value=0.025, step=0.005)),
            activity_regularizer = regularizers.l1(l=hp.Float('l1_reg', min_value=0.015, max_value=0.025, step=0.005))
        ),
        Dropout(rate=0.2, seed = 24),
        layers.Dense(2, activation='softmax')
    ])
    model.compile(
        Adamax(learning_rate=lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=9,
    executions_per_trial=3,
    directory='hype',
    project_name='trd_train') # without hypeparameter of dropout and learning rate after multiple running

tuner.search_space_summary()

tuner.search(datagen.flow(train_images, train_labels, batch_size=32),
             validation_data=(valid_images, valid_labels),
             steps_per_epoch=len(train_images) / 32, epochs=epochs)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
the optimal l1 regularization rate is {best_hps.get('l1_reg')},
the optimal l2 regularization rate is {best_hps.get('l2_reg')},
""")



Trial 8 Complete [00h 55m 35s]
val_accuracy: 0.9537037014961243

Best val_accuracy So Far: 0.9722222089767456
Total elapsed time: 06h 30m 40s
INFO:tensorflow:Oracle triggered exit

the optimal l1 regularization rate is 0.02,
the optimal l2 regularization rate is 0.02,

